<a href="https://colab.research.google.com/github/seyf1elislam/LocalLLM_OneClick_Colab/blob/main/Run_any_gguf_model_in_TextGen_webui.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# $\color{#19ABEA}{\text{Run any gguf quantized models in Latest Version TextGen-webui :}}$
### `version 07/2025`
> changes :  
> add new TextWebUI versions  .  
> add new models to the list  
> add `gpt oss 20B` (supported in v3.9) and you can run it at `Q5_km` and `Q4_k_m`

this [Notebook](https://github.com/seyf1elislam/LocalLLM_OneClick_Colab) allow u run  any quantized gguf file from any Hugging face repository as long as it fits the colab Vram and ram.

>follow this [githubRepository](https://github.com/seyf1elislam/LocalLLM_OneClick_Colab) to get the updates of this noteboook.

> Created by :  
> 😇 github :   [@seyf1elislam](https://github.com/seyf1elislam)  
> 🤗 Huggingface : [@seyf1elislam](https://huggingface.co/seyf1elislam)

## Quantized models Sources :
> you can check those 🤗 huggingface repos :
- [mradermacher](https://huggingface.co/mradermacher)  gguf source ,contains thousand of gguf quantized models.
- [bartowski](https://huggingface.co/bartowski) #contains thousand of gguf quantized models
- [LongStriker](https://huggingface.co/LoneStriker)  gguf source ,contains hundreds of quantized models.(exl2,gguf)
- [QuantFactory](https://huggingface.co/QuantFactory) contains some gguf models.  
- [using search](https://huggingface.co/models?sort=trending&search=gguf) here u can find all gguf files on  huggingface.

## Good models to try :
> you can try those :  
- [QuantFactory/Mistral-Nemo-Instruct-2407-GGUF](https://huggingface.co/QuantFactory/Mistral-Nemo-Instruct-2407-GGUF) 12B model Q5_K_M / Q4_K_M (⭐🔥) .
- [bartowski/Mistral-Small-Instruct-2409-GGUF](https://huggingface.co/bartowski/Mistral-Small-Instruct-2409-GGUF) this is 22B u can use it 3KM in 15g vram (⭐🔥) .
- [Meta-Llama-3.1-8B-Instruct-GGUF](https://huggingface.co/bartowski/Meta-Llama-3.1-8B-Instruct-GGUF) Q8_0 (⭐🔥) .
- [Meta-Llama-3-8B-Instruct-GGUF](https://huggingface.co/QuantFactory/Meta-Llama-3-8B-Instruct-GGUF)  Q8_0  (⭐🔥) .
- [gemma-2-9b-it-GGUF](https://huggingface.co/bartowski/gemma-2-9b-it-GGUF)  Q8_0/Q6   (⭐🔥) .

## Some Tips :
> in free colab gpu T4  (15G vram) you can use :
> - 22b model quantized upto Q3_K_M(`context up to 8K`)
> - 12b model quantized upto Q5_K_M(`context up to 16K`)
> - 8b model quantized upto Q8_0(`context up to 16K or even more if ur model stable with long context`)

> most  models goes with 8k context length if u want to use longer context you need to make sure the model support longer context.

> if you want to run model higher then 20B (such as 20B,4x7b..) on colab you may need to reduce the offloaded gpu layers  to split the ram usage between gpu vram and system ram. (slower but it works 😉)

In [ ]:
#@title # $\color{#19ABEA}{\text{Download and install requirements}}$ {display-mode: "form"}
#@markdown $\color{#19ABEA}{\text{run this only once at the start}}$
%cd /content
## @markdown `Note :no need to check these checkboxes unless u facing a problem`
#@markdown ---
#@markdown `check this if you need to change an installed version only `
delete_existing_texgen_webui_folder =True #@param {type:"boolean"}
if delete_existing_texgen_webui_folder :
  !rm -rf /content/text-generation-webui
#@markdown ---
#@markdown ### TextGen webui version :
#@markdown `v3.9(latest) has been released on 5aug2025`
#@markdown `to get  more info about the newer versions  check the TextGenUI releases here:` https://github.com/oobabooga/text-generation-webui/releases

%cd /content
branch = "v3.9" # @param ["snapshot-2024-01-07","snapshot-2024-01-14","snapshot-2024-01-28","snapshot-2024-02-11","snapshot-2024-02-25","snapshot-2024-03-03","snapshot-2024-03-10","snapshot-2024-03-17","snapshot-2024-03-24","snapshot-2024-03-31","snapshot-2024-04-07","snapshot-2024-04-14","snapshot-2024-04-21","snapshot-2024-04-28","v1.8","v1.12","v1.14","v1.13","v1.15","v3.4","v3.5","v3.6","v3.7","v3.8","v3.9","dev"]
branch = f"-b {branch}"
#@markdown ----
#@markdown `Noe :use this to download the latest version of TextGen Webui use use_latest ,this will ignore the branch/version name and download the latest .`
use_latest =True #@param {type:"boolean"}

if use_latest :
  branch = ""
!git clone {branch} https://github.com/oobabooga/text-generation-webui

%cd /content/text-generation-webui

if branch.startswith("v3") or branch == "":
  !pip install -q -r requirements/full/requirements.txt
else:
  !pip install -q -r requirements.txt

!apt-get -y install -qq aria2
#@markdown ----

from IPython.display import clear_output
clear_output()
print("Done")

In [ ]:
#@title #  $\color{#19ABEA}{Download Model}$ {display-mode: "form"}
#@markdown ## $\color{#19ABEA}{\text{Copy the RepoName here :}}$

#----------
from huggingface_hub import HfFileSystem
fs = HfFileSystem()

def get_donwloadlink_and_filename(repo_name,quant='Q5_K_M'):
  gguf_files = fs.glob(f"{repo_name}/*.gguf")
  filtered_files = [file for file in gguf_files if (quant in file or quant.lower() in file.lower() or quant.upper() in file.upper())]

  if len(filtered_files) ==0 : # quant not existed
    print( "=" * 10 +"\navailable quants:\n",gguf_files)
    return None,None
  file_path= filtered_files[0]
  *repo_name , file_name = file_path.split("/");
  repo_name='/'.join(repo_name)
  return f"https://huggingface.co/{repo_name}/resolve/main/{file_name}?download=true" ,file_name

#----------

#@markdown  Example :`bartowski/Meta-Llama-3.1-8B-Instruct-GGUFF`
repo_name = "unsloth/Qwen3-8B-GGUF" # @param ["QuantFactory/Meta-Llama-3-8B-Instruct-GGUF","bartowski/gemma-2-9b-it-GGUF","QuantFactory/Mistral-Nemo-Instruct-2407-GGUF","bartowski/Mistral-Small-Instruct-2409-GGUF","bartowski/Meta-Llama-3.1-8B-Instruct-GGUF","unsloth/Qwen3-30B-A3B-GGUF","unsloth/gpt-oss-20b-GGUF","unsloth/Qwen3-8B-GGUF","unsloth/Qwen3-14B-GGUF","unsloth/gemma-3-12b-it-GGUF"] {"allow-input":true}

#@markdown in free colab gpu T4  (15G vram) you can use :
#@markdown - 22b model quantized upto Q3_K_M(`context up to 8K`)
#@markdown - 12b model quantized upto Q5_K_M(`context up to 16K`)
#@markdown - 8b model quantized upto Q8_0(`context up to 16K`)

quant ="Q6_K" #@param ['Q2_K', 'Q3_K_L', 'Q3_K_M', 'Q3_K_S', 'Q4_0', 'Q4_1', 'Q4_K_M', 'Q4_K_S', 'Q5_0', 'Q5_1', 'Q5_K_M', 'Q5_K_S', 'Q6_K', 'Q8_0']
model_download_url,model_file_name =get_donwloadlink_and_filename(repo_name,quant=quant)
#@markdown `after v3.5 and later u need to use this user data path`
use_user_data = True  #@param{type:"boolean"}
subpath = ''
if use_user_data:
  subpath ="/user_data"

if model_download_url is not None and model_file_name is not None :
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_download_url} -d /content/text-generation-webui{subpath}/models/ -o {model_file_name}
else :
  print("======================")
  print("please check the repo name or check availability of the quantized file inside the repo")

In [ ]:
#@title ##  $\color{#19ABEA}{RunOnly}$ {display-mode: "form"}
# %cd /content/text-generation-webui
#@markdown  $\color{#19ABEA}{\text{Enter ur Context size or let it unchecked to use the default size}}$
params =" "
#@markdown _____
api =True # @param {type:"boolean"}
if api :params +=" --api --public-api --listen "
api_only_mode_nowebui =False # @param {type:"boolean"}
if api_only_mode_nowebui : params+=" --nowebui "
#@markdown _____
use_custom_context_len = True # @param {type:"boolean"}
context_len="16384" # @param ["4096","8192","12288","16384"] {"allow-input":true}
if use_custom_context_len : params += f" --n_ctx  {context_len} "
#@markdown _____
use_custom_params = True # @param {type:"boolean"}
custom_params = "" # @param {type:"string"}
if use_custom_params : params +=custom_params

#this if u want to use RoPE scale
# params +=" --alpha_value 1.5"

#@markdown _____
# params
!python server.py   --share  --n-gpu-layers 100000  	--model $model_file_name   $params